In [ ]:
#NN for XOR
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)
learning_rate =0.1
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

W = tf.Variable(tf.random_normal([2, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.sigmoid(tf.matmul(X,W)+b)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

#Accuracy Computation

predicted=tf.cast(hypothesis>0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            print('step : ', step, 'cost : ',sess.run(cost, feed_dict={X: x_data, Y: y_data}), 'W : ',sess.run(W))
            
    h,c,a= sess.run([hypothesis,predicted, accuracy],feed_dict={X: x_data, Y: y_data})
    print('hypothesis',h,'\ncorrect : ',c,'\nAccuarcy :',a)

In [1]:
#Wide NN for XOR
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)
learning_rate =0.03
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])
with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2, 4]), name='weight')
    b1 = tf.Variable(tf.random_normal([4]), name='bias')
    layer1 = tf.sigmoid(tf.matmul(X,W1)+b1)

    #step1 decide which tesnors you want to log
    W1_hist= tf.summary.histogram("wieghts2", W1)
    b1_hist= tf.summary.histogram("biases2", b1)
    layer_hist=tf.summary.histogram('hypothesis',layer1)

    
with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([4, 1]), name='weight')
    b2 = tf.Variable(tf.random_normal([1]), name='bias')
    hypothesis = tf.sigmoid(tf.matmul(layer1,W2)+b2)
    
    #step1 decide which tesnors you want to log
    W2_hist= tf.summary.histogram("wieghts2", W2)
    b2_hist= tf.summary.histogram("biases2", b2)
    hypothesis_hist=tf.summary.histogram('hypothesis',hypothesis)

with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    
    #step1 decide which tesnors you want to log
    cost_summ=tf.summary.scalar("cost",cost)
    
with tf.name_scope("train") as scope:
    train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)



#Accuracy Computation
predicted=tf.cast(hypothesis>0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))

with tf.Session() as sess:
    #step2 merge all summaries 
    merged_summary= tf.summary.merge_all()
    #step3 create wirter after creting session
    sess.run(tf.global_variables_initializer())
    writer=tf.summary.FileWriter('./logs/x_or/x_or_logs_0_03')
    writer.add_graph(sess.graph)
    
    
    for step in range(10001):
        s=sess.run(merged_summary, feed_dict={X: x_data, Y: y_data})
        
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        writer.add_summary(s,global_step=step)
        if step % 5000 == 0:
            print('step : ', step, 'cost : ',sess.run(cost, feed_dict={X: x_data, Y: y_data}), 'W : ',sess.run([W1,W2]))
            
    h,c,a= sess.run([hypothesis,predicted, accuracy],feed_dict={X: x_data, Y: y_data})
    print('hypothesis',h,'\ncorrect : ',c,'\nAccuarcy :',a)

step :  0 cost :  0.71885 W :  [array([[ 0.80206424,  0.67859823, -1.21707046, -0.30515447],
       [-0.3029792 ,  1.5083195 ,  0.75680262, -0.70079094]], dtype=float32), array([[ 1.70655143],
       [ 0.3450754 ],
       [-1.13268042],
       [ 1.29066396]], dtype=float32)]
step :  5000 cost :  0.64241 W :  [array([[ 0.37849998,  1.98560488, -1.00357974,  0.27465168],
       [ 0.43456894,  2.44611239, -0.20668861, -0.93219948]], dtype=float32), array([[ 0.93430883],
       [ 1.42377281],
       [-0.78163326],
       [ 1.23773849]], dtype=float32)]
step :  10000 cost :  0.459059 W :  [array([[ 0.41202983,  4.00918293, -0.93294144,  1.3275063 ],
       [ 0.39577577,  4.56074619, -0.55165249, -3.00462914]], dtype=float32), array([[-0.60493904],
       [ 3.86046815],
       [-0.60457265],
       [ 2.54561543]], dtype=float32)]
hypothesis [[ 0.12389518]
 [ 0.52014393]
 [ 0.80844373]
 [ 0.56728303]] 
correct :  [[ 0.]
 [ 1.]
 [ 1.]
 [ 1.]] 
Accuarcy : 0.75


In [ ]:
#Deep NN for XOR
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)
learning_rate =0.1
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

W1 = tf.Variable(tf.random_normal([2, 10]), name='weight')
b1 = tf.Variable(tf.random_normal([10]), name='bias')
layer1 = tf.sigmoid(tf.matmul(X,W1)+b1)

W2 = tf.Variable(tf.random_normal([10, 10]), name='weight')
b2 = tf.Variable(tf.random_normal([10]), name='bias')
layer2 = tf.sigmoid(tf.matmul(layer1,W2)+b2)

W3 = tf.Variable(tf.random_normal([10, 10]), name='weight')
b3 = tf.Variable(tf.random_normal([10]), name='bias')
layer3 = tf.sigmoid(tf.matmul(layer2,W3)+b3)

W4 = tf.Variable(tf.random_normal([10, 1]), name='weight')
b4 = tf.Variable(tf.random_normal([1]), name='bias')
hypothesis = tf.sigmoid(tf.matmul(layer3,W4)+b4)



cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

#Accuracy Computation

predicted=tf.cast(hypothesis>0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 5000 == 0:
            print('step : ', step, 'cost : ',sess.run(cost, feed_dict={X: x_data, Y: y_data}), 'W : ',sess.run([W1,W2]))
            
    h,c,a= sess.run([hypothesis,predicted, accuracy],feed_dict={X: x_data, Y: y_data})
    print('hypothesis',h,'\ncorrect : ',c,'\nAccuarcy :',a)

In [ ]:
# Lab 9
import tensorflow as tf
import random
# import matplotlib.pyplot as plt
#tf.set_random_seed(777)  # for reproducibility

from tensorflow.examples.tutorials.mnist import input_data
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

nb_classes = 10

# MNIST data image of shape 28 * 28 = 784

X = tf.placeholder(tf.float32, [None, 784])
# 0 - 9 digits recognition = 10 classes
Y = tf.placeholder(tf.float32, [None, nb_classes])

with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([784, 256]))
    b1 = tf.Variable(tf.random_normal([256]))
    # Hypothesis (using softmax)
    layer1 = tf.nn.softmax(tf.matmul(X, W1) + b1)
    
with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([256, 256]))
    b2 = tf.Variable(tf.random_normal([256]))
    # Hypothesis (using softmax)
    layer2 = tf.nn.softmax(tf.matmul(layer1, W2) + b2)
    
with tf.name_scope("layer3") as scope:
    W3 = tf.Variable(tf.random_normal([256, nb_classes]))
    b3 = tf.Variable(tf.random_normal([nb_classes]))
    # Hypothesis (using softmax)
    hypothesis = tf.nn.softmax(tf.matmul(layer2, W3) + b3)
    

with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
    cost_summ=tf.summary.scalar("cost",cost)
    
with tf.name_scope("train") as scope:    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.05).minimize(cost)
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(cost)
    #GradientDescent doesn't work well.. I don't why Adam is better than Gradient.

is_correct= tf.equal(tf.arg_max(hypothesis,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32))

training_epochs=15
batch_size=100

with tf.Session() as sess:    
    merged_summary= tf.summary.merge_all()
    #step3 create wirter after creting session
    sess.run(tf.global_variables_initializer())
    writer=tf.summary.FileWriter('./logs/minist/lr_0_05')
    writer.add_graph(sess.graph)
    step=0
    for epoch in range(training_epochs):
        avg_cost=0
        total_batch=int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, optimizer],feed_dict={X: batch_xs, Y:batch_ys})
            avg_cost += c / total_batch
            
            s=sess.run(merged_summary, feed_dict={X: batch_xs, Y: batch_ys})
            writer.add_summary(s,global_step=step)
            step+=1
        
        print("Epoch = ", '%4d'%(epoch+1),'cost = ','{:.9f}'.format(avg_cost))
    # Test the model using test sets
    
    r = random.randint(0, mnist.test.num_examples - 1)
    print("Accuracy: ", accuracy.eval(session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))        
    print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
    print("Prediction: ", sess.run(tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

In [ ]:
 
    for step in range(10001):
        s=sess.run(merged_summary, feed_dict={X: x_data, Y: y_data})
        
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        writer.add_summary(s,global_step=step)